# In this notebook we will get Toronto's postal code with neighborhood information from wikipedia and do data tranformation into pandas dataframe. 

## Part 1 - Listed below the steps taken for data transformation
1. read html date from wiki into pandas dataframe
2. rename the column without space
3. ignore cells with a borough that is Not assigned and reset index
4. if more than one neighborhood exists in one postal code, then those neighborhoods should be combined into one row separated with a comma
5. if a cell has a borough having 'Not assigned' neighborhood, then the neighborhood will be replaced with borough

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# read html date from wiki into pandas dataframe
page = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = pd.DataFrame(page[0])
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
# rename the column without space
df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [4]:
# ignore cells with a borough that is Not assigned and reset index
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df = df.reset_index(drop=True)

In [5]:
# if more than one neighborhood exists in one postal code, then those neighborhoods should be combined into one row separated with a comma
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [6]:
# if a cell has a borough having 'Not assigned' neighborhood, then the neighborhood will be replaced with borough
df['Neighborhood'] = np.where(df.Neighborhood == 'Not assigned', df.Borough, df.Neighborhood)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df.shape

(103, 3)

## Part 2 - Get the latitude and longitude data and merge with the neigborhood information

In [8]:
# read csv data into pandas dataframe and merge
page = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo = pd.DataFrame(page)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# rename the column without space
df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [10]:
# merge the postalcode data with coordination data
df_all = pd.merge(df,df_geo, on='PostalCode')

In [11]:
df_all.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
